In [1]:
#quick script to find the autocorrelation in time of front frequency for each 1 degree latitude bin
#without accounting for autocorrelation, changes in frequency will be found to be more significant
#than they actually are

#a bunch of this code is copied over from RDA_hov_periods.nc and slightly adapted.

In [1]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
from autocorr import autocorr
%matplotlib notebook

In [2]:
#Access NetCDF files (currently hosted in Ferret directory)
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"
RDA_1 = nc.Dataset(RDA_path_1, 'r')
RDA_2 = nc.Dataset(RDA_path_2, 'r')

In [3]:
#load data from NetCDF files to notebook
lat_1_all =  RDA_1.variables['lat_115'][:]
lat_2_all =  RDA_2.variables['lat_115'][:]
intensity_1_all = RDA_1.variables['intensity'][:] 
intensity_2_all = RDA_2.variables['intensity'][:]

RDA_1.close()
RDA_2.close()

In [4]:
#Assign a calendar date to each time point
startday = datetime.datetime(1951,1,1)
date_list = np.array([datetime.timedelta(days=x) + startday for x in range(0, 20819)])

In [5]:
#quick function that bins everything in a 2D array and puts floor and ceiling on array values.
def findbins(lat):
    index = np.ndarray.round(lat-19.5)
    index[index < 0] = 0
    index[index > 21] = 21
    return index

In [6]:
#bin latitudes
lat_1_indices = findbins(lat_1_all)
lat_2_indices = findbins(lat_2_all)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater


In [7]:
#create a time series of whether a rainband exists for each latitude bin
occupancy_1 = np.zeros([20819,22])
occupancy_2 = np.zeros([20819,22])

for i, index in enumerate(lat_1_indices):
    if ~np.isnan(index):
        occupancy_1[i,index] = 1
        
for i, index in enumerate(lat_2_indices):
    if ~np.isnan(index):
        occupancy_2[i,index] = 1

#also create combined occupancy matrix of both primary and secondary fronts
occupancy = occupancy_1 + occupancy_2

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [8]:
def smooth(P,dayrange,latrange):
    Psmooth=np.zeros(P.shape)
    
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    for d in range(days):
        for j in range(lats):
            
            Psample = P.take(range(d-ll,d+ll+1), mode='wrap', axis=0)
            Ps = Psample[:,max(0,j-yy):min(days,j+yy+1)]
            Psmooth[d,j] = np.mean(np.mean(Ps))
    
    return Psmooth

In [43]:
#switching it to binary - hence the *5. Wanted to keep same def of smooth function as elsewhere
#NOTE: in some cases, values can reach 2 (if both fronts are within small latitude range on a given day)
#should be a minimal effect - we test both keeping the 2 and reducing to 1.
occupancy_1_smth = smooth(occupancy_1,1,5)*5
occupancy_2_smth = smooth(occupancy_2,1,5)*5
occupancy_smth = smooth(occupancy,1,5)*5


In [44]:
#as constructed, occupancy_smth can have values of 2 if fronts are close enough - this fixes that.
occupancy_smth_bin = np.copy(occupancy_smth) #without using copy, still points to same array.
occupancy_smth_bin[occupancy_smth_bin>1] = 1

In [45]:
#quick comparison of how much the smoothing in latitude changes mean front occupancy
print(np.mean(occupancy, axis=0))
print(np.mean(occupancy_smth, axis=0))

[ 0.00048033  0.00086459  0.00384264  0.02877179  0.02656227  0.03227821
  0.03443969  0.03818627  0.04303761  0.03669725  0.03247034  0.02987656
  0.01902109  0.01998175  0.01565877  0.01508238  0.01196023  0.00979874
  0.00667659  0.00614823  0.00403478  0.00134493]
[ 0.00864595  0.04244921  0.06052164  0.09231952  0.12589462  0.16023824
  0.17450406  0.18463903  0.18483116  0.18026802  0.16110284  0.13804698
  0.1170085   0.09962054  0.08170421  0.07248187  0.05917671  0.04966617
  0.03861857  0.02800327  0.02275566  0.01921322]


In [46]:
for row1, row2 in zip(occupancy_smth.T, occupancy_smth_bin.T):
    print(autocorr(row1))
    print(autocorr(row2))
    time.sleep(2)
    

(array([ 1.        ,  0.07855679,  0.03201813,  0.00409276,  0.01340204,
        0.00409265,  0.0040926 , -0.00521813,  0.00409249,  0.00409244,
       -0.00521963]), 1.2680042668501552)
(array([ 1.        ,  0.07855679,  0.03201813,  0.00409276,  0.01340204,
        0.00409265,  0.0040926 , -0.00521813,  0.00409249,  0.00409244,
       -0.00521963]), 1.268004266850155)
(array([ 1.        ,  0.18447634,  0.08198325,  0.04684087,  0.03951928,
        0.03512599,  0.03512599,  0.02633814,  0.024873  ,  0.03659077,
        0.02194231]), 2.0656318846028086)
(array([ 1.        ,  0.18447634,  0.08198325,  0.04684087,  0.03951928,
        0.03512599,  0.03512599,  0.02633814,  0.024873  ,  0.03659077,
        0.02194231]), 2.0656318846028086)
(array([ 1.        ,  0.19915678,  0.08595211,  0.04117345,  0.0530015 ,
        0.04877604,  0.04286014,  0.03778873,  0.03778745,  0.03017988,
        0.02172639]), 2.1968049501985747)
(array([ 1.        ,  0.19915678,  0.08595211,  0.04117345,  0.053

KeyboardInterrupt: 

In [9]:
#Define empty variables reshaped with axis 0 as day of year and axis 1 as year.
lat_1 = np.zeros([365, 57])
lat_2 = np.zeros([365, 57])
int_1 = np.zeros([365, 57])
int_2 = np.zeros([365, 57])
        
for i, date in enumerate(date_list[0:365]): #go through each day of the year

    mymap = map(lambda date: (date.day == dd) & (date.month == mm), date_list)
    extract = np.array(list(mymap))

    lat_1[i,:] = lat_1_all[extract]
    lat_2[i,:] = lat_2_all[extract]
    int_1[i,:] = intensity_1_all[extract]
    int_2[i,:] = intensity_2_all[extract]